In [2]:
import numpy as np
import pandas as pd
import keras
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

In [3]:
import common.constant as const
from gomoku.player import Player
from gomoku.gomoku import Gomoku
from common.exception import AlreadyPlacedExcpetion

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
gomoku = Gomoku()
player1 = Player('Player 1', 'B')
player2 = Player('Player 2', 'W')
gomoku.start(player1, player2, render = False)

In [5]:
# Initialising the ANN
model = Sequential()

model.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu', input_dim = 675))
model.add(Dropout(rate = 0.1))

model.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(rate = 0.1))

model.add(Dense(units = 384, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(rate = 0.1))

model.add(Dense(units = 256, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(rate = 0.1))

model.add(Dense(units = 225, kernel_initializer = 'uniform', activation = 'softmax'))

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = opt)

In [6]:
df = pd.DataFrame(columns = ['iteration', 'max_step', 'black_win_rate', 'white_win_rate', 'valid_click_rate'])

In [9]:
now = datetime.now()
t_str = now.strftime("%Y-%m-%d %H-%M-%S")

max_iter = 10000
max_turn = 10000

black_win_count = 0
white_win_count = 0

verbose = 0

for iteration in range(1, 10000):
    
    print('Iteration: {}/{}:'.format(iteration, max_iter))

    gomoku.restart()
    
    turn = 'B'
    player1.place(7, 7)
    turn = 'W'
    success = 0
    valid_click_rate = 0


    for i in range(1, max_turn):
        if turn == 'B':
            curr_player = player1
            opponent_player = player2
            
        else:
            curr_player = player2
            opponent_player = player1
        
        X = curr_player.map_my_pieces.reshape((1, 225))
        X = np.append(X, opponent_player.map_my_pieces.reshape((1, 225)), axis = 1)
        X = np.append(X, curr_player.map_prev_move.reshape((1, 225)), axis = 1)
        
        y_pred = model.predict(X)
        
        # pos = y_pred.argmax()
        pos = y_pred.argmax()
        r = pos // 15
        c = pos % 15
        
        try:
            result = curr_player.place(r, c)
            if result == const.PLACE_SUCCESS:
                turn = opponent_player.color
                success = success + 1
                
            elif result == const.PLACE_WIN:
                y_reward = curr_player.map_prev_move.reshape((1, 225))
                model.fit(X, y_reward, verbose = verbose)
                
                m_pos = y_reward.argmax()
                m_r = m_pos // 15
                m_c = m_pos % 15
                
                print('Rewarding ', r, c, m_r, m_c)
                
                if curr_player.color == 'B':
                    black_win_count = black_win_count + 1
                else:
                    white_win_count = white_win_count + 1
                
                break
                
        except AlreadyPlacedExcpetion:
            y_reward = curr_player.map_prev_move.reshape((1, 225)) * -1
            model.fit(X, y_reward, verbose = verbose)
        
        if i % 10 == 0:
            valid_click_rate = success / i
            print(' ' * 100, end = '\r')
            print('{}/{} Step: {} Current attempt: ({}, {}) Valid click rate: {:.2f}'.format(i, max_turn, success, r, c, valid_click_rate), end = '\r')
    
    print(' ' * 100, end = '\r')
    print('{} won! Max step: {} Current attempt: ({}, {}) Valid click rate: {:.2f}'.format(curr_player.name, success, r, c, valid_click_rate))
    
    df.at[iteration - 1, 'iteration'] = iteration
    df.at[iteration - 1, 'max_step'] = success
    df.at[iteration - 1, 'black_win_rate'] = black_win_count / iteration
    df.at[iteration - 1, 'white_win_rate'] = white_win_count / iteration
    df.at[iteration - 1, 'valid_click_rate'] = valid_click_rate
    
    gomoku.save('{} {}'.format(t_str, iteration))
    
    if iteration % 1 == 0:
        df.to_csv('./statistics/{}.csv'.format(t_str))

Iteration: 1/10000:
Rewarding  3 3 3 32 Current attempt: (8, 13) Valid click rate: 0.20                                 
Player 1 won! Max step: 113 Current attempt: (3, 3) Valid click rate: 0.20                          
Iteration: 2/10000:
Rewarding  3 7 3 7 Current attempt: (2, 13) Valid click rate: 0.20                                  
Player 1 won! Max step: 91 Current attempt: (3, 7) Valid click rate: 0.20                           
Iteration: 3/10000:
Rewarding  9 8 9 84 Current attempt: (7, 12) Valid click rate: 0.23                                 
Player 2 won! Max step: 144 Current attempt: (9, 8) Valid click rate: 0.23                          
Iteration: 4/10000:


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x000002866809E5E0>
Traceback (most recent call last):
  File "C:\Users\Bowen WU\anaconda3\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 534, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\Users\Bowen WU\anaconda3\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1262, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


Rewarding  14 11 14 11rrent attempt: (13, 7) Valid click rate: 0.18                                 
Player 1 won! Max step: 109 Current attempt: (14, 11) Valid click rate: 0.18                        
Iteration: 5/10000:


KeyboardInterrupt: 

In [ ]:
gomoku.save()

In [8]:
df.head(5)

,iteration,max_step,black_win_rate,white_win_rate,valid_click_rate
0,1,1,0,0,0.0001001
1,2,1,0,0,0.0001001
2,3,82,0,1,0.218919
3,4,107,0.25,0.75,0.246512
4,5,173,0.4,0.6,0.2


In [16]:
import concurrent.futures

def foo(bar):
    print('hello {}'.format(bar))
    return 'foo'

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(foo, 'world!')
    future2 = executor.submit(foo, 'world2!')
    return_value = future.result()
    print(return_value)
    return_value2 = future2.result()
    print(return_value2)

hello world!
foo
hello world2!
foo
